# Use Documents to tune generation without a vector store
This notebook show an example of Retrieval Augmented Generation that utilizes LangChain to perform question-answering tasks by combining retrieval and generation techniques.

## What is Retrieval Augmented Generation?
Retrieval Augmented Generation (RAG)'s purpose is to increase the relevance, accuracy and truthfulness of generation. This way we remove the "data freshness" problem that LLM's inherently have.

It allows you to retrieve relevant documents based on a query and use them as context to generate concise answers to user questions.

## Dependencies

The Retrieval Augmented Generation module relies on the following dependencies:

- `langchain`: The core LangChain library for building the pipeline and running the modules.

- `langchain_community`: The LangChain community package that provides additional modules and utilities.

- `langchain_core`: The core components and utilities of LangChain.

In [1]:
%pip install -Uq langchain openai unstructured chardet

Note: you may need to restart the kernel to use updated packages.


### Enable LangChain tracing (Optional)

In [2]:
import os
os.environ['LANGSMITH_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = "unstructuredfileloader"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ""

In [3]:
import chardet

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders.unstructured import UnstructuredFileLoader
from unstructured.cleaners.core import clean_extra_whitespace

# Set API Key
os.environ['OPENAI_API_KEY'] = "sk-"

In [5]:
# Step 1: Define the retriever function using UnstructuredFileLoader
loader = UnstructuredFileLoader(
    "./data/france.txt", post_processors=[clean_extra_whitespace],
    )
# The files contents: "The Capital of France is Paris."
docs = loader.load()

In [6]:
docs[0].page_content[:40]

'The Capital of France is Paris.'

In [7]:
# Step 2: Define the prompt template with the file loader as the context
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template, context=docs)

In [8]:
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [9]:
# Step 3: Create the LangChain pipeline
llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)
output_parser = StrOutputParser()

chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [10]:
# Step 4: Invoke the LangChain pipeline with a question
question = "What is the capital of France?"
answer = chain.invoke({"question": question})

In [11]:
print(answer)

The capital of France is Paris.


## Explanation
In this example, the Retrieval Augmented Generation module is used to answer a user's question about the capital of France. The module retrieves relevant documents based on the query using the `UnstructuredFileLoader` and incorporates them into the prompt template. 

The LangChain pipeline is then created by chaining together the retriever, prompt, language model (LLM), and output parser components. Finally, the pipeline is invoked with the user's question, and the generated answer is printed.

## Customization

The Retrieval Augmented Generation module can be customized to fit your specific needs. Here are some areas to consider:

• Modify the prompt template to structure the prompt according to your requirements. You can include placeholders for the question, retrieved context, or any other information you want to provide to the language model.

• Use different language models by specifying the desired model name when creating the `ChatOpenAI` instance. You can explore different models provided by OpenAI, open-source models on the HuggingFace Hub, or use your own fine-tuned models.

• Customize the output parser to parse the generated answer in a format that suits your application's needs.

## Conclusion

The Retrieval Augmented Generation module provides a convenient way to perform question-answering tasks by combining retrieval and generation techniques. It allows you to retrieve relevant documents based on a query and generate concise answers using LangChain. With its customization options, you can tailor the module to suit your specific needs and integrate it into your applications for enhanced question-answering capabilities.